Before running: 

In [ ]:
! pip install numpy sklearn torchvision comet_ml

In [ ]:
from comet_ml import Experiment  # has to be 1st import
import numpy as np
import sklearn
from sklearn.svm import SVC
from torchvision import datasets, transforms

# Setup

In [ ]:
# INSERT YOUR DATA HERE
# Expected format: One folder per class, e.g.
# train

In [ ]:
# |   +-- lassie.jpg
# |   +-- komissar-rex.png

In [ ]:
# |   +-- garfield.png
# |   +-- smelly-cat.png
train_data = "path/to/data/dir"  # required
val_data = None                  # optional
test_data = None                 # optional

# Set up logging.
experiment = Experiment("")


# # Preprocessing


def preprocess(data, name):
    if data is None:  # val/test can be empty
        return None

    # Read image files to pytorch dataset (only temporary).
    transform = transforms.Compose(
        [transforms.Resize(28), transforms.ToTensor()]
    )
    data = datasets.ImageFolder(data, transform=transform)
    
    # Convert to numpy arrays.
    images_shape = (len(data), *data[0][0].shape)
    images = np.zeros(images_shape)
    labels = np.zeros(len(data))
    for i, (image, label) in enumerate(data):
        images[i] = image
        labels[i] = label

    # Flatten.
    images = images.reshape(len(images), -1)

    # Shuffle train set.
    if name == "train":
        images, labels = sklearn.utils.shuffle(images, labels)

    return [images, labels]

processed_train_data = preprocess(train_data, "train")
processed_val_data = preprocess(val_data, "val")
processed_test_data = preprocess(test_data, "test")


# # Model


model = SVC()


# # Training


def evaluate(data, name):
    if data is None:  # val/test can be empty
        return

    images, labels = data
    acc = model.score(images, labels)
    print(f"{name} accuracy:".ljust(18) + str(acc))
    experiment.log_metric(f"{name}_accuracy", acc)

# Train on train_data.
model.fit(*processed_train_data)

# Evaluate on all datasets.
evaluate(processed_train_data, "train")
evaluate(processed_val_data, "val")
evaluate(processed_test_data, "test")